In [1]:
import os
os.chdir("..")
print("Current Directory:", os.getcwd())

Current Directory: d:\workspace\iscat


In [7]:
os.path.basename( "data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\zd\Brightfield.png")

'Brightfield.png'

In [8]:
for dirpath, dirnames, filenames in os.walk(data_path):
    if os.path.basename(dirpath) == 'Brightfield': 
        print("Current Path:", dirpath)
        print("Directories:", dirnames)
        print("Files:", filenames)
        for file in filenames:
            if file.endswith('.nd2'): 
                print(os.path.join(dirpath, file))

Current Path: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield
Directories: []
Files: ['02_01_BF.nd2', '02_01_BF_frame_000.tiff', '02_01_BF_frame_100.tiff', '02_01_BF_frame_200.tiff', 'Captured Cy5.csv', 'Captured Cy5.tif', 'Captured FITC.csv', 'Captured FITC.tif', 'Captured TRITC.csv', 'Captured TRITC.tif', 'Cy5.csv', 'FITC.csv', 'mask.npy', 'TRITC.csv']
data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\02_01_BF.nd2
Current Path: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield
Directories: []
Files: ['02_02_BF.nd2', '02_02_BF_frame_000.tiff', '02_02_BF_frame_100.tiff', '02_02_BF_frame_200.tiff', 'Captured Cy5.csv', 'Captured Cy5.tif', 'Captured FITC.csv', 'Captured FITC.tif', 'Captured TRITC.csv', 'Captured TRITC.tif', 'Cy5.csv', 'FITC.csv', 'mask.npy', 'TRITC.csv']
data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\02_02_BF.nd2
Current Path: data\iScat\Data\2024_11_11\Metasurface\Chip_02

In [2]:
from src.data_processing.utils import Utils
data_path = os.path.join('data', 'iScat', 'Data', '2024_11_11', 'Metasurface', 'Chip_02')
image_paths,target_paths = Utils.get_data_paths(data_path)

Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\frame_0.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\frame_100.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\frame_200.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\frame_0.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\frame_100.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\frame_200.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 03\Brightfield\frame_0.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 03\Brightfield\frame_100.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 03\Brightfield\frame_200.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 04\Brightfield\frame_0.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Ch

In [8]:
class Us:
    @staticmethod
    def get_data_paths(root_path, mode="Brightfield", image_indices=[0,100,200]):
        """
        Extract paths to .nd2 files and corresponding TIFF files from the specified mode folder.

        Args:
            root_path (str): The root directory to search.
            mode (str): The folder name to focus on (default is 'Brightfield').

        Returns:
            tuple: Two lists - list of .nd2 file paths and list of tuples with corresponding TIFF file paths.
        """
        zimages_files = []
        target_files = []
        
        for dirpath, dirnames, filenames in os.walk(root_path):
            if os.path.basename(dirpath) == mode:  # Focus on the specified mode folder
                for indices in image_indices:
                    base_path = os.path.join(dirpath, f"frame_{indices}.tiff")
                    zimages_files.append(base_path)
                if  not all(os.path.exists(path) for path in zimages_files):  
                    Us.process_nd2_file(dirpath,image_indices)

                # Generate TIFF file paths dynamically based on the prefix
                cy5_path = os.path.join(dirpath, f'Captured Cy5.tif')
                fitc_path = os.path.join(dirpath, f'Captured FITC.tif')
                tritc_path = os.path.join(dirpath, f'Captured TRITC.tif')
                target_files.append((cy5_path, fitc_path, tritc_path))
                
                # Ensure all three TIFF files exist
                assert all(os.path.exists(path) for path in [cy5_path, fitc_path, tritc_path])
                assert all(os.path.exists(path) for path in zimages_files)                 
                
        
        return zimages_files, target_files
    @staticmethod
    def process_nd2_file(file_path,indices):
    # Open the .nd2 file
        with nd2.ND2File(file_path) as f:
            array = f.asarray()  # Load the entire stack as a NumPy array
            num_frames = array.shape[0]  # Number of frames
            # Save each specified frame as a 16-bit TIFF
            for idx in indices:
                assert 0 <= idx < num_frames
                frame = array[idx]
                # Generate the output path
                output_path = os.path.join(
                    os.path.dirname(file_path), 
                    f"{os.path.splitext(os.path.basename(file_path))[0]}_frame_{idx:03d}.tiff"
                )
                # Save the frame as a 16-bit TIFF
                imwrite(output_path, frame, dtype='uint16')
                print(f"Saved: {output_path}")
 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 26)

In [6]:
target_paths

[('data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 01\\Brightfield\\Captured Cy5.tif',
  'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 01\\Brightfield\\Captured FITC.tif',
  'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 01\\Brightfield\\Captured TRITC.tif'),
 ('data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 02\\Brightfield\\Captured Cy5.tif',
  'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 02\\Brightfield\\Captured FITC.tif',
  'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 02\\Brightfield\\Captured TRITC.tif'),
 ('data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 03\\Brightfield\\Captured Cy5.tif',
  'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 03\\Brightfield\\Captured FITC.tif',
  'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 03\\Brightfield\\Captured TRITC.tif'),
 ('data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_

In [3]:
image_paths

['data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 01\\Brightfield\\02_01_BF.nd2',
 'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 02\\Brightfield\\02_02_BF.nd2',
 'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 03\\Brightfield\\02_03_BF.nd2',
 'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 04\\Brightfield\\02_04_BF.nd2',
 'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 05\\Brightfield\\02_05_BF.nd2']

In [4]:
import os
import nd2  # Install via `pip install nd2reader`
from tifffile import imwrite  # Install via `pip install tifffile`
indices = [0,100,200]
def process_nd2_file(file_path,indices):
    # Open the .nd2 file
    with nd2.ND2File(file_path) as f:
        array = f.asarray()  # Load the entire stack as a NumPy array
        num_frames = array.shape[0]  # Number of frames
        # Save each specified frame as a 16-bit TIFF
        for idx in indices:
            assert 0 <= idx < num_frames
            frame = array[idx]
            # Generate the output path
            output_path = os.path.join(
                os.path.dirname(file_path), 
                f"{os.path.splitext(os.path.basename(file_path))[0]}_frame_{idx:03d}.tiff"
            )
            # Save the frame as a 16-bit TIFF
            imwrite(output_path, frame, dtype='uint16')
            print(f"Saved: {output_path}")
 

In [7]:
with nd2.ND2File(image_paths[0]) as f:
    array = f.asarray() 

In [10]:
array.shape

(201, 2160, 2560)

In [11]:
for file in image_paths:
    process_nd2_file(file)

Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\02_01_BF_frame_000.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\02_01_BF_frame_100.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 01\Brightfield\02_01_BF_frame_200.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\02_02_BF_frame_000.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\02_02_BF_frame_100.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 02\Brightfield\02_02_BF_frame_200.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 03\Brightfield\02_03_BF_frame_000.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 03\Brightfield\02_03_BF_frame_100.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\Metasurface 03\Brightfield\02_03_BF_frame_200.tiff
Saved: data\iScat\Data\2024_11_11\Metasurface\Chip_02\M